In [1]:
print('welcome')

welcome


In [12]:
import pymongo
from pymongo import MongoClient
from datetime import datetime, timedelta, timezone
import pandas as pd
from faker import Faker


client = MongoClient("mongodb://localhost:27017/")
db = client['eduhub_db']

In [ ]:
import random
from bson import ObjectId
from bson.objectid import ObjectId
import pprint




In [4]:
# 1. USERS Collection Validation
users_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["userId", "email", "firstName", "lastName", "role", "dateJoined", "isActive"],
        "properties": {
            "userId": {"bsonType": "string"},
            "email": {"bsonType": "string", "pattern": "^.+@.+$"},
            "firstName": {"bsonType": "string"},
            "lastName": {"bsonType": "string"},
            "role": {"enum": ["student", "instructor"]},
            "dateJoined": {"bsonType": "date"},
            "profile": {
                "bsonType": "object",
                "properties": {
                    "bio": {"bsonType": "string"},
                    "avatar": {"bsonType": "string"},
                    "skills": {
                        "bsonType": "array",
                        "items": {"bsonType": "string"}
                    }
                }
            },
            "isActive": {"bsonType": "bool"}
        }
    }
}
db.create_collection("users", validator=users_validator)


# 2. COURSES Collection Validation
courses_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["courseId", "title", "instructorId", "level", "duration", "price", "createdAt", "updatedAt", "isPublished"],
        "properties": {
            "courseId": {"bsonType": "string"},
            "title": {"bsonType": "string"},
            "description": {"bsonType": "string"},
            "instructorId": {"bsonType": "string"},
            "category": {"bsonType": "string"},
            "level": {"enum": ["beginner", "intermediate", "advanced"]},
            "duration": {"bsonType": "double"},
            "price": {"bsonType": "double"},
            "tags": {
                "bsonType": "array",
                "items": {"bsonType": "string"}
            },
            "createdAt": {"bsonType": "date"},
            "updatedAt": {"bsonType": "date"},
            "isPublished": {"bsonType": "bool"}
        }
    }
}
db.create_collection("courses", validator=courses_validator)


# 3. ENROLLMENTS Collection Validation
enrollments_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["enrollmentId", "studentId", "courseId", "enrolledAt", "status"],
        "properties": {
            "enrollmentId": {"bsonType": "string"},
            "studentId": {"bsonType": "string"},
            "courseId": {"bsonType": "string"},
            "enrolledAt": {"bsonType": "date"},
            "status": {"enum": ["in progress", "completed", "dropped"]},
            "progress": {"bsonType": "double"}  # percentage (0–100)
        }
    }
}
db.create_collection("enrollments", validator=enrollments_validator)


# 4. LESSONS Collection Validation
lessons_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["lessonId", "courseId", "title", "content", "createdAt"],
        "properties": {
            "lessonId": {"bsonType": "string"},
            "courseId": {"bsonType": "string"},
            "title": {"bsonType": "string"},
            "content": {"bsonType": "string"},
            "duration": {"bsonType": "double"},
            "createdAt": {"bsonType": "date"}
        }
    }
}
db.create_collection("lessons", validator=lessons_validator)


# 5. ASSIGNMENTS Collection
assignments_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["assignmentId", "courseId", "title", "description", "dueDate", "createdAt"],
        "properties": {
            "assignmentId": {"bsonType": "string"},
            "courseId": {"bsonType": "string"},
            "title": {"bsonType": "string"},
            "description": {"bsonType": "string"},
            "dueDate": {"bsonType": "date"},
            "createdAt": {"bsonType": "date"}
        }
    }
}
db.create_collection("assignments", validator=assignments_validator)


# 6. SUBMISSIONS Collection
submissions_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["submissionId", "assignmentId", "studentId", "submittedAt"],
        "properties": {
            "submissionId": {"bsonType": "string"},
            "assignmentId": {"bsonType": "string"},
            "studentId": {"bsonType": "string"},
            "submittedAt": {"bsonType": "date"},
            "content": {"bsonType": "string"},
            "grade": {"bsonType": "double"},
            "feedback": {"bsonType": "string"}
        }
    }
}
db.create_collection("submissions", validator=submissions_validator)


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'eduhub_db'), 'submissions')

In [21]:
# Initialize Faker
fake = Faker()

# Helper function to generate unique IDs
def gen_id(prefix):
    import uuid
    return f"{prefix}_{uuid.uuid4().hex[:8]}"

def random_choice(choices):
    return random.choice(choices)


In [ ]:

# Create 25 users
users = []
for _ in range(25):
    user = {
        "userId": gen_id("user"),
        "email": fake.email(),
        "firstName": fake.first_name(),
        "lastName": fake.last_name(),
        "role": random_choice(["student", "instructor"]),
        "dateJoined": fake.date_time_between(start_date='-1y', end_date='now'),
        "isActive": random.choice([True, False]),
        "profile": {
            "bio": fake.text(100),
            "avatar": fake.image_url(),
            "skills": fake.words(nb=random.randint(1, 5))
        }
    }
    users.append(user)
db.users.insert_many(users)

# Insert data into the COURSES collection
courses = []
for i in range(8):
    instructor = random.choice([u for u in users if u["role"] == "instructor"])
    now = datetime.now(timezone.utc)
    course = {
        "courseId": gen_id("course"),
        "title": fake.sentence(3),
        "description": fake.paragraph(nb_sentences=3),
        "instructorId": instructor["userId"],
        "category": fake.word(),
        "level": random_choice(["beginner", "intermediate", "advanced"]),
        "duration": round(random.uniform(1.0, 20.0), 1),
        "price": round(random.uniform(10.0, 200.0), 2),
        "tags": fake.words(nb=random.randint(2, 5)),
        "createdAt": now,
        "updatedAt": now,
        "isPublished": random.choice([True, False])
    }
    courses.append(course)
db.courses.insert_many(courses)

# Insert data into the ENROLLMENTS courses
enrollments = []
for _ in range(15):
    student = random.choice([u for u in users if u["role"] == "student"])
    course = random.choice(courses)
    enrollment = {
        "enrollmentId": gen_id("enr"),
        "studentId": student["userId"],
        "courseId": course["courseId"],
        "enrolledAt": fake.date_time_between(start_date='-6mo', end_date='now'),
        "status": random_choice(["in progress", "completed", "dropped"]),
        "progress": round(random.uniform(0, 100), 2)
    }
    enrollments.append(enrollment)
db.enrollments.insert_many(enrollments)

#Insert data into the LESSONS collection 
lessons = []
lesson_count = 25
course_ids = [course["courseId"] for course in courses]

for _ in range(lesson_count):
    course_id = random.choice(course_ids)  # randomly assign lesson to an existing course
    lesson = {
        "lessonId": gen_id("les"),
        "courseId": course_id,
        "title": fake.sentence(4),
        "content": fake.text(300),
        "duration": round(random.uniform(5, 60), 2),
        "createdAt": fake.date_time_between(start_date='-6mo', end_date='now')
    }
    lessons.append(lesson)

db.lessons.insert_many(lessons)





InsertManyResult([ObjectId('684ddb59cbd1ecd6b45645ae'), ObjectId('684ddb59cbd1ecd6b45645af'), ObjectId('684ddb59cbd1ecd6b45645b0'), ObjectId('684ddb59cbd1ecd6b45645b1'), ObjectId('684ddb59cbd1ecd6b45645b2'), ObjectId('684ddb59cbd1ecd6b45645b3'), ObjectId('684ddb59cbd1ecd6b45645b4'), ObjectId('684ddb59cbd1ecd6b45645b5'), ObjectId('684ddb59cbd1ecd6b45645b6'), ObjectId('684ddb59cbd1ecd6b45645b7'), ObjectId('684ddb59cbd1ecd6b45645b8'), ObjectId('684ddb59cbd1ecd6b45645b9'), ObjectId('684ddb59cbd1ecd6b45645ba'), ObjectId('684ddb59cbd1ecd6b45645bb'), ObjectId('684ddb59cbd1ecd6b45645bc'), ObjectId('684ddb59cbd1ecd6b45645bd'), ObjectId('684ddb59cbd1ecd6b45645be'), ObjectId('684ddb59cbd1ecd6b45645bf'), ObjectId('684ddb59cbd1ecd6b45645c0'), ObjectId('684ddb59cbd1ecd6b45645c1'), ObjectId('684ddb59cbd1ecd6b45645c2'), ObjectId('684ddb59cbd1ecd6b45645c3'), ObjectId('684ddb59cbd1ecd6b45645c4'), ObjectId('684ddb59cbd1ecd6b45645c5'), ObjectId('684ddb59cbd1ecd6b45645c6'), ObjectId('684ddb59cbd1ecd6b45645

In [24]:
# Create 10 assignments
def gen_id(prefix):
    return f"{prefix}_{uuid.uuid4().hex[:8]}"

assignments = []
for i in range(10):
    course = random.choice(courses)  # Randomly assign to any existing course
    now = datetime.now(timezone.utc)
    assignment = {
        "assignmentId": gen_id("asg"),
        "courseId": course["courseId"],
        "title": fake.sentence(3),
        "description": fake.text(150),
        "dueDate": now + timedelta(days=random.randint(5, 30)),
        "createdAt": now
    }
    assignments.append(assignment)
db.assignments.insert_many(assignments)
print("✅ Inserted 10 assignments")

#Create 12 submissions
students = [u for u in users if u["role"] == "student"]
submissions = []
for _ in range(12):
    assignment = random.choice(assignments)
    student = random.choice(students)
    submission = {
        "submissionId": gen_id("sub"),
        "assignmentId": assignment["assignmentId"],
        "studentId": student["userId"],
        "submittedAt": fake.date_time_between(start_date='-30d', end_date='now'),
        "content": fake.text(200),
        "grade": round(random.uniform(0, 100), 1),
        "feedback": fake.sentence()
    }
    submissions.append(submission)
db.submissions.insert_many(submissions)
print("✅ Inserted 12 assignment submissions")

✅ Inserted 10 assignments
✅ Inserted 12 assignment submissions


In [14]:
# Basic CRUD operator
# Create Operation

# Create a new user 
new_student = {
    "userId": gen_id("user"),
    "email": "adeolu1@example.com",
    "firstName": "Adeolu",
    "lastName": "Olorunwa",
    "role": "student",
    "dateJoined": datetime.now(timezone.utc),
    "isActive": True,
    "profile": {
        "bio": "I am passionate about learning new technologies.",
        "avatar": "https://example.com/avatar.jpg",
        "skills": ["Python", "SQL", "Data Analysis"]
    }
}
db.users.insert_one(new_student)
print("✅ New student added:", new_student["userId"])


✅ New student added: user_3baae9d2


In [15]:
#Create a new course
instructor = db.users.find_one({"role": "instructor"})
if instructor:
    now = datetime.now(timezone.utc)
    new_course = {
        "courseId": gen_id("course"),
        "title": "Introduction to Data Science",
        "description": "This course provides a beginner-friendly introduction to data science concepts.",
        "instructorId": instructor["userId"],
        "category": "Data Science",
        "level": "beginner",
        "duration": 15.0,
        "price": 120.0,
        "tags": ["data", "python", "machine learning"],
        "createdAt": now,
        "updatedAt": now,
        "isPublished": True
    }
    db.courses.insert_one(new_course)

In [16]:
#Enroll a student in a course
course = db.courses.find_one()

if course:
    new_lesson = {
        "lessonId": gen_id("les"),
        "courseId": course["courseId"],
        "title": "Getting Started with Python",
        "content": "In this lesson, we will install Python and write our first program.",
        "duration": 45.0,
        "createdAt": datetime.now(timezone.utc)
    }
    db.lessons.insert_one(new_lesson)


In [17]:
#Add a new lesson to an existing course
course = db.courses.find_one()

if course:
    new_lesson = {
        "lessonId": gen_id("les"),
        "courseId": course["courseId"],
        "title": "Getting Started with SQL",
        "content": "In this lesson, we will install SQL and write our first program.",
        "duration": 45.0,
        "createdAt": datetime.now(timezone.utc)
    }
    db.lessons.insert_one(new_lesson)


In [ ]:
# Read operation: Find all active students
active_students = list(db.users.find({
    "role": "student",
    "isActive": True
}))

# Convert to DataFrame
df = pd.DataFrame(active_students)

# Drop the MongoDB _id column for readability
if '_id' in df.columns:
    df = df.drop(columns=['_id'])

# Display the DataFrame
print(df)

           userId                         email  firstName  lastName     role  \
0   user_5428d83f  melindarodriguez@example.org     Rachel     Baker  student   
1   user_d27f9f98     melissabowers@example.com  Stephanie     Smith  student   
2   user_db2b69ed         heather44@example.org   Savannah     Lynch  student   
3   user_0d36e495       kylejackson@example.org   Jennifer     Hogan  student   
4   user_2ae6eef2          zbeasley@example.net     Janice   Holland  student   
5   user_412ab980            ndavis@example.com     Tammie     Boyer  student   
6   user_621d1a3c         melissa29@example.net     Robert    Dorsey  student   
7   user_c2efd78e      jacqueline15@example.org      James   Patrick  student   
8   user_ed8edf6f         fwilliams@example.com      Diana      Frey  student   
9   user_71cbce6d      cherylhoward@example.com    Michael   Houston  student   
10  user_c1c371c8          agarrett@example.com   Kimberly  Erickson  student   
11  user_780569a8      grego

In [9]:
#Retrieve course details with instructor information
courses_with_instructors = db.courses.aggregate([
    {
        "$lookup": {
            "from": "users",
            "localField": "instructorId",
            "foreignField": "userId",
            "as": "instructor"
        }
    }
])

for course in courses_with_instructors:
    # Check if instructor list is not empty
    if course["instructor"]:
        instructor = course["instructor"][0]  # get the first (and should be only) instructor
        print(f'{course["title"]}: {instructor["firstName"]} {instructor["lastName"]} (Instructor ID: {instructor["userId"]})')
    else:
        print(f'{course["title"]}: Instructor not found')



Movie picture.: Ashley Stevens (Instructor ID: user_a9e8ce43)
Instead college whom office.: Ashley Stevens (Instructor ID: user_a9e8ce43)
Lay movie drug serve.: Tonya Floyd (Instructor ID: user_916b8d45)
Change page main.: Ashley Stevens (Instructor ID: user_a9e8ce43)
About he change.: Leslie Brown (Instructor ID: user_a65f952c)
Party reveal daughter.: Leslie Brown (Instructor ID: user_a65f952c)
Though behind.: Samantha Kelley (Instructor ID: user_0482efeb)
Must strong.: Bryan Hill (Instructor ID: user_560e2df6)
Introduction to Data Science: Mario Anderson (Instructor ID: user_e7ab5a50)


In [28]:
#Get all courses in a specific category
category = "movie" 
courses = list(db.courses.find({"category": category}))
for course in courses:
    print(course["title"], "-", course["category"])

About he change. - movie


In [32]:
#Find students enrolled in a particular course
course_id = "course_44ba7402" 

# Get all studentIds for the course
enrollments = db.enrollments.find({"courseId": course_id})

student_ids = [enr["studentId"] for enr in enrollments]

# Query users collection for those students
students = list(db.users.find({
    "userId": {"$in": student_ids},
    "role": "student"
}))

for student in students:
    print(student["firstName"], student["lastName"])
else:
    print("Not found")


Amanda Mcneil
Kiara Thomas
Glen Casey
Not found


In [33]:
#Search courses by title (case-insensitive, partial match)
search_text = "Change"  # Partial word or phrase to search

matched_courses = db.courses.find({
    "title": {
        "$regex": search_text,
        "$options": "i"  # case-insensitive
    }
})

for course in matched_courses:
    print(course["title"])

Change page main.
About he change.


In [ ]:
#Update Operations
#Update a user’s profile information
user_id = "user_914b4f4a"

db.users.update_one(
    { "userId": user_id },
    {
        "$set": {
            "profile.bio": "Experienced Data Analyst.",
            "profile.skills": ["Python", "Data Analysis", "SQL"]
        }
    }
)


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [37]:
#Mark a course as published
db.courses.update_one(
    { "courseId": "course_777faaa3" },
    { "$set": { "isPublished": True } }
)


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [43]:
#Update assignment grades
db.submissions.update_one(
    { "submissionId": "sub_20c63da8" },
    { "$set": { "grade": 92.5 } }
)
submit = db.submissions.find_one( { "submissionId": "sub_20c63da8" },{"grade":1})
print(submit)

{'_id': ObjectId('684ddb59cbd1ecd6b45645af'), 'grade': 92.5}


In [44]:
# Add tags to an existing course
course_id = "course_37773db4"

db.courses.update_one(
    { "courseId": course_id },
    { "$addToSet": { "tags": { "$each": ["statistics", "visualization"] } } }
)


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [ ]:
#Delete Operations
#Remove a user (soft delete by setting isActive to false)
user_id = "user_914b4f4a"  

db.users.update_one(
    { "userId": user_id },
    { "$set": { "isActive": False } }
)

UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [46]:
#Delete an enrollment
enrollment_id = "enr_4b6f45bb"

db.enrollments.delete_one(
    { "enrollmentId": enrollment_id }
)

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [47]:
#Remove a lesson from a course
lesson_id = "les_7855227d"

db.lessons.delete_one(
    { "lessonId": lesson_id }
)

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [49]:
#Complex Queries
#Find courses with price between $50 and $200
courses_in_range = db.courses.find({
    "price": {
        "$gte": 50,
        "$lte": 200
    }
})

for course in courses_in_range:
    print(course["title"], ": $", course["price"])

Movie picture. : $ 166.34
Instead college whom office. : $ 193.64
Change page main. : $ 121.15
Party reveal daughter. : $ 147.29
Though behind. : $ 158.04
Must strong. : $ 75.53
Introduction to Data Science : $ 120.0


In [10]:
#Get users who joined in the last 6 months
# Calculate date 6 months ago
six_months_ago = datetime.now() - timedelta(days=180)

# Query users who joined in the last 6 months
recent_users_cursor = db.users.find({
    "dateJoined": { "$gte": six_months_ago }
})

# Convert cursor to list and then DataFrame
recent_users_list = list(recent_users_cursor)
df = pd.DataFrame(recent_users_list)


# Display the result
print(df)

                         _id         userId                         email  \
0   684dd980cbd1ecd6b456450b  user_5428d83f  melindarodriguez@example.org   
1   684dd980cbd1ecd6b456450c  user_4b0f06ea   mitchelljeffrey@example.net   
2   684dd980cbd1ecd6b456450e  user_e7ab5a50        lhernandez@example.com   
3   684dd980cbd1ecd6b4564510  user_b397a453         jeffrey98@example.com   
4   684dd980cbd1ecd6b4564514  user_6e3e2b15      danielsuarez@example.com   
5   684dd980cbd1ecd6b4564515  user_c6dc6955     richardsutton@example.org   
6   684dd980cbd1ecd6b4564516  user_879a912d   kimberlywilkins@example.org   
7   684dd980cbd1ecd6b4564518  user_d27f9f98     melissabowers@example.com   
8   684dd980cbd1ecd6b4564519  user_2499f90d            dawn42@example.com   
9   684dd980cbd1ecd6b456451a  user_1f878511           jwilson@example.com   
10  684dd980cbd1ecd6b456451c  user_2c701e20  wallacechristine@example.com   
11  684dd980cbd1ecd6b456451d  user_335150dd             ncook@example.com   

In [53]:
#Find courses that have specific tags using $in operator
tags = ["leader","yourself"]

tagged_courses = db.courses.find({
    "tags": { "$in": tags }
})

for course in tagged_courses:
    print(course["title"], "tags:", course["tags"])


Must strong. tags: ['leader', 'yourself', 'budget', 'huge']


In [54]:
#Retrieve assignments with due dates in the next week
now = datetime.now()
next_week = now + timedelta(days=7)

upcoming_assignments = db.assignments.find({
    "dueDate": {
        "$gte": now,
        "$lte": next_week
    }
})

for assignment in upcoming_assignments:
    print(assignment["title"], "due on", assignment["dueDate"])

Both into. due on 2025-06-21 20:28:09.678000


In [59]:
#Aggregation Pipeline
#Course Enrollment Statistics
Total_enrollments = db.enrollments.aggregate([
    {"$group": {
        "_id": "$courseId",
        "totalEnrollments": {"$sum": 1}
    }}
])
results = list(Total_enrollments)
print(results)


[{'_id': 'course_37773db4', 'totalEnrollments': 4}, {'_id': 'course_6b823027', 'totalEnrollments': 1}, {'_id': 'course_4e260ccf', 'totalEnrollments': 3}, {'_id': 'course_28851d34', 'totalEnrollments': 1}, {'_id': 'course_44ba7402', 'totalEnrollments': 3}, {'_id': 'course_777faaa3', 'totalEnrollments': 2}]


In [62]:
#Calculate average course rating
avg_course_ratings = list(db.enrollments.aggregate([
    {"$match": {"rating": {"$exists": True}}},
    {"$group": {
        "_id": "$courseId",
        "avgRating": {"$avg": "$rating"}
    }}
]))

print(avg_course_ratings)


[]


In [ ]:
#Group by course category
course_category = db.courses.aggregate([
    {"$group": {
        "_id": "$category",
        "totalCourses": {"$sum": 1}
    }}
])
for course in course_category:
    print(f"Category: {course['_id']} => Total Courses: {course['totalCourses']}")


Category: movie => Total Courses: 1
Category: foot => Total Courses: 1
Category: total => Total Courses: 1
Category: way => Total Courses: 1
Category: agent => Total Courses: 1
Category: that => Total Courses: 1
Category: Data Science => Total Courses: 1
Category: surface => Total Courses: 1
Category: center => Total Courses: 1


In [67]:
avg_grades = db.submissions.aggregate([
    {
        "$group": {
            "_id": "$studentId",
            "avgGrade": {"$avg": "$grade"}
        }
    }
])

for student in avg_grades:
    print(f"Student ID: {student['_id']} = Average Grade: {round(student['avgGrade'], 2)}")


Student ID: user_459944d1 = Average Grade: 63.67
Student ID: user_30a4008a = Average Grade: 58.33
Student ID: user_af693f0d = Average Grade: 46.02
Student ID: user_dd0dbc72 = Average Grade: 55.01
Student ID: user_28dd95f2 = Average Grade: 55.98
Student ID: user_31d41563 = Average Grade: 50.91
Student ID: user_8829b32c = Average Grade: 39.52
Student ID: user_4899b9e2 = Average Grade: 39.52
Student ID: user_6f2a7997 = Average Grade: 55.89
Student ID: user_3ab3a989 = Average Grade: 50.61
Student ID: user_08bf22ef = Average Grade: 55.02
Student ID: user_38ae84c6 = Average Grade: 57.64
Student ID: user_25431486 = Average Grade: 56.76


In [72]:
# Completion rate by course
completion_rates = db.enrollments.aggregate([
    {
        "$group": {
            "_id": "$courseId",
            "completed": {"$sum": {"$cond": [{"$eq": ["$status", "completed"]}, 1, 0]}},
            "total": {"$sum": 1}
        }
    },
    {
        "$project": {
            "completionRate": {"$divide": ["$completed", "$total"]}
        }
    }
])



In [73]:
#Top-performing students
Top_performing = db.submissions.aggregate([
    {"$group": {
        "_id": "$studentId",
        "avgGrade": {"$avg": "$grade"}
    }},
    {"$sort": {"avgGrade": -1}},
    {"$limit": 5}
]
)
result = list(Top_performing)
print (result)

[{'_id': 'user_459944d1', 'avgGrade': 63.67142857142857}, {'_id': 'user_30a4008a', 'avgGrade': 58.333333333333336}, {'_id': 'user_38ae84c6', 'avgGrade': 57.6375}, {'_id': 'user_25431486', 'avgGrade': 56.7625}, {'_id': 'user_28dd95f2', 'avgGrade': 55.98}]


In [74]:
#Instructor Analytics:
#Total students taught by each instructor
db.courses.aggregate([
    {"$lookup": {
        "from": "enrollments",
        "localField": "courseId",
        "foreignField": "courseId",
        "as": "enrollments"
    }},
    {"$unwind": "$enrollments"},
    {"$group": {
        "_id": "$instructorId",
        "studentsTaught": {"$addToSet": "$enrollments.studentId"}
    }},
    {"$project": {
        "instructorId": "$_id",
        "uniqueStudents": {"$size": "$studentsTaught"}
    }}
])


In [75]:
#Average course rating per instructor
db.courses.aggregate([
    {"$lookup": {
        "from": "enrollments",
        "localField": "courseId",
        "foreignField": "courseId",
        "as": "enrollments"
    }},
    {"$unwind": "$enrollments"},
    {"$match": {"enrollments.rating": {"$exists": True}}},
    {"$group": {
        "_id": "$instructorId",
        "avgRating": {"$avg": "$enrollments.rating"}
    }}
])



In [76]:
# Revenue generated per instructor
revenue_per_instructor = db.courses.aggregate([
    {
        "$lookup": {
            "from": "enrollments",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "enrollments"
        }
    },
    {
        "$project": {
            "instructorId": 1,
            "price": 1,
            "enrollmentCount": {"$size": "$enrollments"}
        }
    },
    {
        "$group": {
            "_id": "$instructorId",
            "totalRevenue": {"$sum": {"$multiply": ["$price", "$enrollmentCount"]}}
        }
    }
])

# Print results
for instructor in revenue_per_instructor:
    print(f"Instructor ID: {instructor['_id']} - Total Revenue: ${round(instructor['totalRevenue'], 2)}")


Instructor ID: user_a65f952c - Total Revenue: $147.29
Instructor ID: user_916b8d45 - Total Revenue: $46.56
Instructor ID: user_0482efeb - Total Revenue: $158.04
Instructor ID: user_560e2df6 - Total Revenue: $0.0
Instructor ID: user_e7ab5a50 - Total Revenue: $0.0
Instructor ID: user_a9e8ce43 - Total Revenue: $1277.05


In [77]:
#Advanced Analytics:
# Monthly enrollment trends
monthly_enrollments = db.enrollments.aggregate([
    {
        "$group": {
            "_id": {
                "year": {"$year": "$enrolledAt"},
                "month": {"$month": "$enrolledAt"}
            },
            "totalEnrollments": {"$sum": 1}
        }
    },
    {
        "$sort": {"_id.year": 1, "_id.month": 1}
    }
])

# Print results
print("📅 Monthly Enrollment Trends:")
for doc in monthly_enrollments:
    print(f"{doc['_id']['year']}-{doc['_id']['month']:02d} => {doc['totalEnrollments']} enrollments")


📅 Monthly Enrollment Trends:
2025-06 => 14 enrollments


In [78]:
# Most popular course categories
popular_categories = db.courses.aggregate([
    {
        "$group": {
            "_id": "$category",
            "courseCount": {"$sum": 1}
        }
    },
    {
        "$sort": {"courseCount": -1}
    }
])

# Print results
print("\n🏆 Most Popular Course Categories:")
for category in popular_categories:
    print(f"Category: {category['_id']} => {category['courseCount']} courses")




🏆 Most Popular Course Categories:
Category: surface => 1 courses
Category: movie => 1 courses
Category: center => 1 courses
Category: agent => 1 courses
Category: that => 1 courses
Category: Data Science => 1 courses
Category: way => 1 courses
Category: foot => 1 courses
Category: total => 1 courses


In [80]:
# Student engagement metrics
engagement = db.submissions.aggregate([
    {
        "$group": {
            "_id": "$studentId",
            "submissionCount": {"$sum": 1},
            "averageGrade": {"$avg": "$grade"}
        }
    },
    {
        "$sort": {"submissionCount": -1}
    }
])

# Print results
print("Student Engagement Metrics:")
for student in engagement:
    print(f"Student ID: {student['_id']} => Submissions: {student['submissionCount']}, Avg Grade: {round(student['averageGrade'], 2)}")


Student Engagement Metrics:
Student ID: user_30a4008a => Submissions: 9, Avg Grade: 58.33
Student ID: user_6f2a7997 => Submissions: 9, Avg Grade: 55.89
Student ID: user_31d41563 => Submissions: 9, Avg Grade: 50.91
Student ID: user_38ae84c6 => Submissions: 8, Avg Grade: 57.64
Student ID: user_4899b9e2 => Submissions: 8, Avg Grade: 39.52
Student ID: user_dd0dbc72 => Submissions: 8, Avg Grade: 55.01
Student ID: user_af693f0d => Submissions: 8, Avg Grade: 46.02
Student ID: user_25431486 => Submissions: 8, Avg Grade: 56.76
Student ID: user_3ab3a989 => Submissions: 7, Avg Grade: 50.61
Student ID: user_459944d1 => Submissions: 7, Avg Grade: 63.67
Student ID: user_08bf22ef => Submissions: 6, Avg Grade: 55.02
Student ID: user_28dd95f2 => Submissions: 5, Avg Grade: 55.98
Student ID: user_8829b32c => Submissions: 5, Avg Grade: 39.52


In [81]:
#Index Creation
#User email lookup
db.users.create_index("email", unique=True)


'email_1'

In [82]:
#Course search by title and category
db.courses.create_index([("title", "text"), ("category", 1)])


'title_text_category_1'

In [83]:
#Assignment queries by due date
db.assignments.create_index("dueDate")


'dueDate_1'

In [84]:
#Enrollment queries by student and course
db.enrollments.create_index([("studentId", 1), ("courseId", 1)])


'studentId_1_courseId_1'

In [86]:
#Analyze query performance using explain() method in PyMongo
db.users.find_one({"email": "student@example.com"})


In [87]:
#Optimize at least 3 slow queries
db.courses.find({ "$text": { "$search": "Python" } })


In [89]:
#Document the performance improvements using Python timing functions
db.assignments.find({ "dueDate": { "$lt": datetime.now() } })


In [ ]:

#Schema Validation
db.create_collection("users", validator={
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["userId", "email", "firstName", "lastName", "role", "dateJoined", "isActive"], #Required fields
        "properties": {
            "userId": {"bsonType": "string"},
            "email": {
                "bsonType": "string",
                "pattern": "^\\S+@\\S+\\.\\S+$",  # Regex for email format
                "description": "Must be a valid email address"
            },
            "firstName": {"bsonType": "string"},
            "lastName": {"bsonType": "string"},
            "role": {
                "bsonType": "string",
                "enum": ["student", "instructor"],  # Enum restriction
                "description": "Must be either 'student' or 'instructor'"
            },
            "dateJoined": {"bsonType": "date"},
            "isActive": {"bsonType": "bool"}
        }
    }
})

print("✅ Schema validation set up for 'users' collection.")


In [95]:
##Error Handling
#Duplicate key errors
from pymongo.errors import WriteError

from pymongo.errors import DuplicateKeyError

try:
    db.users.insert_one({
        "userId": "user_5428d93f",
        "email": "martinezpaul@example.org",
        "firstName": "Hassan",
        "lastName": "Isah",
        "role": "student",
        "dateJoined": datetime.now(),
        "isActive": True
    })
except DuplicateKeyError:
    print(" Duplicate email detected.")



 Duplicate email detected.


In [96]:

#Invalid data type insertions
try:
    db.users.insert_one({
        "userId": "user_5428d93f",
        "email": "wrongtype@email.com",
        "firstName": "Grace",
        "lastName": "Abah",
        "role": "student",
        "dateJoined": "not a date",
        "isActive": True
    })
except WriteError as e:
    print("WriteError due to invalid data type:", e)

WriteError due to invalid data type: Document failed validation, full error: {'index': 0, 'code': 121, 'errmsg': 'Document failed validation', 'errInfo': {'failingDocumentId': ObjectId('684e90b3cbd1ecd6b4564616'), 'details': {'operatorName': '$jsonSchema', 'schemaRulesNotSatisfied': [{'operatorName': 'properties', 'propertiesNotSatisfied': [{'propertyName': 'dateJoined', 'details': [{'operatorName': 'bsonType', 'specifiedAs': {'bsonType': 'date'}, 'reason': 'type did not match', 'consideredValue': 'not a date', 'consideredType': 'string'}]}]}]}}}


In [97]:
#Missing required fields
try:
    db.users.insert_one({
        # "userId" is missing 
        "email": "missing@field.com",
        "firstName": "Amaka",
        "lastName": "Okoro",
        "role": "instructor",
        "dateJoined": datetime.now(),
        "isActive": True
    })
except WriteError as e:
    print("Missing required field:", e)

Missing required field: Document failed validation, full error: {'index': 0, 'code': 121, 'errmsg': 'Document failed validation', 'errInfo': {'failingDocumentId': ObjectId('684e9139cbd1ecd6b4564617'), 'details': {'operatorName': '$jsonSchema', 'schemaRulesNotSatisfied': [{'operatorName': 'required', 'specifiedAs': {'required': ['userId', 'email', 'firstName', 'lastName', 'role', 'dateJoined', 'isActive']}, 'missingProperties': ['userId']}]}}}
